In [6]:

%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [7]:
# CELL 1

import SPOCK.long_term_scheduler as SPOCKLT
import SPOCK.short_term_scheduler as SPOCKST
import SPOCK.plots_scheduler as SPOCKplot
from astropy.time import Time
import pandas as pd
from astropy.utils import iers
import matplotlib.pyplot as plt
iers.IERS_A_URL  ='ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
import yaml
import sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output, Javascript
from plotly import graph_objs as go
from IPython.display import Image
import os
from astropy import units as u
import datetime
from astroplan import Observer
%matplotlib notebook
from astropy.coordinates import SkyCoord,EarthLocation,CartesianRepresentation
import markdown 
from ipywidgets import HTML
import warnings
warnings.filterwarnings('ignore')

#from astroplan import download_IERS_A
#download_IERS_A()


In [8]:
# CELL 2
observatory_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose site',
    options=['SSO', 'SNO', 'Saint-Ex', 'TS','TN'],
    description='Site:',
    ensure_option=True,
    disabled=False)

telescope_widget = widgets.Combobox(
    # value='John',
    placeholder='SSO telescope',
    options=['Io','Europa','Ganymede','Callisto'],
    description='Telescope:',
    ensure_option=True,
    disabled=False)

operator_widget = widgets.Combobox(
    # value='John',
    placeholder='Opertor for TS/TN',
    options=['Manu','Youssef','Fran','Khalid','Marin'],
    description='Operator:',
    ensure_option=True,
    disabled=False)

start_date_widget =widgets.DatePicker(
    description='Start date',
    disabled=False)

end_date_widget =widgets.DatePicker(
    description='End date',
    disabled=False)

input_file_widget = widgets.Text(description='File',placeholder='./SPOCK/input.csv')

run_widget = widgets.Button(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

short_term_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose option',
    options=['Follow-up','Special target with known start/end time',\
             'Special target (observe the most we can)','Dome rotation','Monitoring'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

short_day_widget =widgets.DatePicker(
    description='Day ',
    disabled=False)

short_start_time_widget = widgets.Text(description='Start time',placeholder='2020-01-16 23:00:00')
short_end_time_widget = widgets.Text(description='End time',placeholder='2020-01-17 03:00:00')

start_time_widget =widgets.DatePicker(
    description='Start Time',
    disabled=False)

end_time_widget =widgets.DatePicker(
    description='End Time',
    disabled=False)

input_file_special_widget = widgets.Text(description='File',placeholder='/Users/elsaducrot/spock/SPOCK/input_short_term.csv')
 
input_target_list_widget = widgets.Text(description='File',placeholder='./SPOCK/speculoos_target_list_v6.txt')
input_target_list_widget.value = './SPOCK/speculoos_target_list_v6.txt'
    
target_widget = widgets.Combobox(
    # value='John',
    placeholder='Spxxx-xxxx',
    options=list(pd.read_csv('./SPOCK/speculoos_target_list_v6.txt',delimiter=' ')['Sp_ID']), 
    description='Target name:',
    ensure_option=True,
    disabled=False)

target_special_widget = widgets.Combobox(
    # value='John',
    placeholder='Trappist-1x',
    options=list(pd.read_csv('./SPOCK/target_list_special.txt',delimiter=' ')['Sp_ID']), 
    description='Target name:',
    ensure_option=True,
    disabled=False)

plot_option_widget = widgets.Combobox(
    # value='John',
    placeholder='Choose a plot option',
    options=['Gant chart','Visibility  plot','Global gant chart'],
    description='What kind? :',
    ensure_option=True,
    disabled=False)

def run_all(b): 
    display(Javascript('Jupyter.notebook.execute_cells([3])'))


# ----------------------------------------------------------- #
    
long_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        widgets.HBox([start_date_widget,end_date_widget]), \
                                        input_file_widget,\
                                       run_widget])
long_term_tab.set_title(0, 'Observatory')
long_term_tab.set_title(1, 'Date range')
long_term_tab.set_title(2, 'Input file')
long_term_tab.set_title(3, 'Run long term')

# ----------------------------------------------------------- #

short_term_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget,operator_widget]),\
                                        input_file_special_widget,widgets.VBox([widgets.HBox([short_term_option_widget,short_day_widget]),\
                                        widgets.HBox([target_special_widget,short_start_time_widget,short_end_time_widget])]),run_widget])
short_term_tab.set_title(0, 'Observatory')
short_term_tab.set_title(1, 'Input file')
short_term_tab.set_title(2, 'Option')
short_term_tab.set_title(3, 'Run short term')

# ----------------------------------------------------------- #

plot_tab = widgets.Accordion(children=[input_file_widget,plot_option_widget,input_target_list_widget,run_widget])
plot_tab.set_title(0, 'Input file')
plot_tab.set_title(1, 'Plot option')
plot_tab.set_title(2, 'Target list')
plot_tab.set_title(3, 'Run plots')

# ----------------------------------------------------------- #

make_upload_tab = widgets.Accordion(children=[widgets.HBox([observatory_widget,telescope_widget]),\
                                          widgets.HBox([start_date_widget,end_date_widget]),\
                                          run_widget])
make_upload_tab.set_title(0, 'Site info')
make_upload_tab.set_title(1, 'Date info')
make_upload_tab.set_title(2, 'Go to plan maker and upload')


# ----------------------------------------------------------- #

run_widget.on_click(run_all)

tab_nest = widgets.Tab()
tab_nest.children = [long_term_tab, short_term_tab,plot_tab,make_upload_tab]
tab_nest.set_title(0, 'Long term scheduler ')
tab_nest.set_title(1, 'Short term scheduler')
tab_nest.set_title(2, 'Plots')
tab_nest.set_title(3, 'Save & Upload only')
tab_nest

<IPython.core.display.Javascript object>

In [10]:
# CELL 3

if tab_nest.selected_index == 0:
    if input_file_widget.value is '':
        input_file = './SPOCK/input.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS':
            obs = 4
        if observatory_widget.value == 'TN':
            obs = 5
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')
    else:
        start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters(input_file,obs)
        schedule.date_range = Time([start_date,end_date])
        if observatory_widget.value == 'SSO':
            schedule.telescope = str(telescope_widget.value)
        schedule.make_schedule(Altitude_constraint = 25, Moon_constraint = 30)
        display(schedule.target_table_spc[schedule.idx_first_target])
        display(schedule.target_table_spc[schedule.idx_second_target])
        display(Javascript('Jupyter.notebook.execute_cells([6])'))

if tab_nest.selected_index == 1:
    if input_file_widget.value is '':
        input_file = './SPOCK/input_short_term.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS':
            obs = 4
        if observatory_widget.value == 'TN':
            obs = 5

    schedule = SPOCKST.Schedules()
    schedule.load_parameters(input_file,obs)
    if observatory_widget.value == 'SSO':
        schedule.telescope = str(telescope_widget.value)
    schedule.use = short_term_option_widget.value
    schedule.day_of_night = Time(datetime.datetime.combine(short_day_widget.value,datetime.time(15,0,0)),format='datetime')
    #schedule.day_of_night = Time(schedule.day_of_night.iso)
    
    if short_term_option_widget.value == 'Follow-up':
        schedule.transit_follow_up('./SPOCK/target_transit_follow_up.txt')
    if short_term_option_widget.value == 'Special target with known start/end time':
        start_date = [Time(short_start_time_widget.value).iso]
        end_date = [Time(short_end_time_widget.value).iso]
        schedule.start_end_range = Time([start_date,end_date])
        input_name = target_special_widget.value
        schedule.special_target_with_start_end(input_name)
    if short_term_option_widget.value == 'Special target (observe the most we can)':
        input_name = target_special_widget.value
        schedule.special_target(input_name)
    if short_term_option_widget.value == 'Dome rotation':
        schedule.dome_rotation()
    if short_term_option_widget.value == 'Monitoring':
        start_date = Time(short_start_time_widget.value).iso
        end_date = Time(short_end_time_widget.value).iso
        schedule.start_end_range = Time([start_date,end_date])
        input_name = target_special_widget.value
        schedule.monitoring(input_name,5,61)
    schedule.make_scheduled_table()
    schedule.planification()
    schedule.make_night_block()
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
    SPOCKST.make_plans(day=schedule.day_of_night,nb_days=1,telescope=schedule.telescope)
    display(schedule.scheduled_table_sorted)
    display(Javascript('Jupyter.notebook.execute_cells([6])'))
    
if tab_nest.selected_index == 2:   
    if plot_option_widget.value == 'Visibility  plot':
        display(Javascript('Jupyter.notebook.execute_cells([4])'))

    if plot_option_widget.value == 'Global gant chart':
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters('./SPOCK/input.csv',1)
        SPOCKplot.gantt_chart_all(schedule.target_list)
        
    if plot_option_widget.value == 'Gant chart':
        display(Javascript("Jupyter.notebook.execute_cells([5])"))

if tab_nest.selected_index == 3:
    if input_file_widget.value is '':
        input_file = './SPOCK/input.csv'
    else:
        input_file = input_file_widget.value
    if observatory_widget.value is '':
        sys.exit('ERROR: You must choose a site')
    else:
        if observatory_widget.value == 'SSO':
            obs = 1
        if observatory_widget.value == 'SNO':
            obs = 2
        if observatory_widget.value == 'Saint-Ex':
            obs = 3
        if observatory_widget.value == 'TS':
            obs = 4
        if observatory_widget.value == 'TN':
            obs = 5
    if start_date_widget.value is '' and end_date_widget.value is '':
        sys.exit('ERROR: The start and/or end date are not valid')
    else:
        start_date = Time(datetime.datetime.strptime(str(start_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        end_date = Time(datetime.datetime.strptime(str(end_date_widget.value), '%Y-%m-%d')+datetime.timedelta(hours=15)).iso
        schedule = SPOCKLT.Schedules()
        schedule.load_parameters(input_file,obs)
        schedule.date_range = Time([start_date,end_date])
        if observatory_widget.value == 'SSO':
            schedule.telescope = str(telescope_widget.value)
    display(Javascript("Jupyter.notebook.execute_cells([6])"))


<IPython.core.display.Javascript object>

In [11]:
# CELL 4
@interact
def plt_target(Site=['Saint-Ex','SSO','SNO','TS_La_Silla','TN_Oukaimeden'],Target=list(pd.read_csv(input_target_list_widget.value,delimiter=' ')['Sp_ID']),day_widget = widgets.DatePicker(description='Pick a Date')):
    if Site== 'SSO':
        obs = 1
    if Site== 'SNO':
        obs = 2
    if Site== 'Saint-Ex':
        obs = 3
    if Site== 'TS_La_Silla':
        obs = 4
    if Site== 'TN_Oukaimeden':
        obs = 5
    schedule = SPOCKLT.Schedules()
    schedule.load_parameters(input_file,obs)
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.airmass_altitude_plot_given_target(name_observatory=Site,day=day,target=Target,path_target_list=input_target_list_widget.value)

    except TypeError:
        print('ERROR: Choose a date ! ')

interactive(children=(Dropdown(description='Site', options=('Saint-Ex', 'SSO', 'SNO', 'TS_La_Silla', 'TN_Oukai…

In [ ]:
# CELL 5
@interact
def plt_gant_chart(Telescope=['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex'],start_widget = widgets.DatePicker(description='Start Date'),end_widget = widgets.DatePicker(description='End Date')):
    try:
        day = Time(datetime.datetime.combine(day_widget,datetime.time(15,0,0)),format='datetime')
        SPOCKplot.gantt_chart(day,day+1,['Europa','Io','Callisto','Ganymede','Artemis','Saint-Ex'])
    except TypeError:
        print('ERROR: Choose a date ! ')

In [ ]:
# CELL 6
save_widget = widgets.Button(
    value=False,
    description='Save plans',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save',
    icon='check'
)
def run_save(b): 
    display(Javascript('Jupyter.notebook.execute_cells([7])'))

display(save_widget)

save_widget.on_click(run_save)


In [ ]:
# CELL 7
if tab_nest.selected_index == 0:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        SPOCKLT.upload_plans(start_date, nb_days=schedule.date_range_in_days,telescope = schedule.telescope)

    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
              is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'SSO':
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'TSlasilla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
              is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'TNOuka':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.save_schedule(None,obs,True,True,schedule.date_range,schedule.telescope)
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
if tab_nest.selected_index == 1:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'Saint-Ex':
        schedule.telescope = 'Saint-Ex'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'SSO':
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_np(schedule.day_of_night,1,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
              is the case and you want access please send a request to educrot@uliege.be.')

    if schedule.observatory.name == 'TNOuka':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKST.save_schedule(None,obs,True,True,schedule.day_of_night,schedule.telescope)
        SPOCKST.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
              is the case and you want access please send a request to educrot@uliege.be.')

if tab_nest.selected_index == 3:
    schedule.telescope = telescope_widget.value
    if schedule.observatory.name == 'SNO':
        schedule.telescope = 'Artemis'
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'saintex':
        schedule.telescope = 'Saint-Ex'
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
                
    if schedule.observatory.name == 'SSO':
        SPOCKLT.make_np(start_date,schedule.date_range_in_days,schedule.telescope)
        print('WARNING: You must be part of the SPECULOOS consortium to access SPECULOOS control PCs. If this \
                      is the case and you want access please send a request to educrot@uliege.be.')
        
    if schedule.observatory.name == 'TS_La_Silla':
        schedule.telescope = 'TS_La_Silla'
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
      is the case and you want access please send a request to educrot@uliege.be.')

    if schedule.observatory.name == 'TN_Oukaimeden':
        schedule.telescope = 'TN_Oukaimeden'
        SPOCKLT.make_docx_schedule(schedule.observatory,schedule.telescope,schedule.date_range,operator_widget.value,schedule.target_list)
        print('WARNING: You must be part of the SPECULOOS consortium to access TRAPPIST control PCs. If this \
              is the case and you want access please send a request to educrot@uliege.be.')
